# Reading and Writing Audio Files with scipy.io

[back to overview page](https://github.com/mgeier/python-audio/blob/master/audio-files/README.md)

The package `scipy.io` is part of [SciPy](http://www.scipy.org/), a library for scientific computing, which itself depends heavily on [NumPy](http://www.numpy.org/).

Documentation: http://docs.scipy.org/doc/scipy/reference/io.html

Audio data is stored in NumPy arrays, which is convenient.
Audio channels are represented by the columns of the array.

16-bit files are stored with data type `int16`, normalization to the range (-1, 1) must be done manually.

Advantages:

* already installed if you have SciPy
* 32-bit float files supported (since version 0.13, see https://github.com/scipy/scipy/commit/ccbdff8)
* can read some WAVEX files (but a warning message may be generated)
* works with both Python 2 and 3

Disadvantages:

* 24-bit PCM not supported
* 32-bit float WAVEX files don't work
* conversion from PCM data to floating point and normalization must be done manually
* always loads whole file, cannot read part of a file
* needs NumPy and SciPy (which is normally not a problem)

## Reading

Reading a 16-bit WAV file into a floating-point array is simple, here's the summary:

In [ ]:
from scipy.io import wavfile
from utility import pcm2float

fs, sig = wavfile.read('data/test_wav_pcm16.wav')
normalized = pcm2float(sig, 'float32')

But let's do that step-by-step, shall we? First, let's use inline plotting and import all the nice NumPy stuff:

In [ ]:
%pylab inline

All relevant functions are in the `wavfile` module from the `scipy.io` package. That's all we have to import for reading and writing WAV files:

In [ ]:
from scipy.io import wavfile

Now let's open a WAV file, get some information about it, show its actual sample values and plot it:

In [ ]:
fs, sig = wavfile.read('data/test_wav_pcm16.wav')
print("sampling rate = {} Hz, length = {} samples, channels = {}".format(fs, *sig.shape))
print(sig)
plot(sig);

So far, so good.

Before further processing, we normally want to convert the signals to floating point values and normalize them to a range from -1 to 1 by dividing all values by the largest possible value.

To do that, I wrote a little helper function called `pcm2float()`, located in the file [`utility.py`](https://github.com/mgeier/python-audio/blob/master/audio-files/utility.py), let's load it:


In [ ]:
from utility import pcm2float

As always, you can get help with `pcm2float?` and show its source code with `pcm2float??`.

In [ ]:
print("old dtype: {}".format(sig.dtype))
normalized = pcm2float(sig, 'float32')
print("new dtype: {}".format(normalized.dtype))
set_printoptions(precision=6)
print(normalized)
plot(normalized);

That's it! Now we have a floating point signal with values ranging from -1 to 1 and we can start working with it.
If you prefer double precision numbers instead of single precision, use `'float64'` instead of `'float32'`.

Let's check if WAVEX works (there might be a warning):

In [ ]:
fs, sig = wavfile.read('data/test_wavex_pcm16.wav')
plot(sig);

24-bit files may raise a `TypeError` ("data type not understood") or a `ValueError` ("total size of new array must be unchanged") or another `ValueError` ("string size must be a multiple of element size") or probably something else.
This depends on the version of SciPy (and maybe Python itself).

In [ ]:
import traceback

try:
    fs, sig = wavfile.read('data/test_wav_pcm24.wav')
except:
    traceback.print_exc()
else:
    print("Surprisingly, wav_pcm24 seems to work!")
    plot(sig)

try:
    fs, sig = wavfile.read('data/test_wavex_pcm24.wav')
except:
    traceback.print_exc()
else:
    print("Surprisingly, wavex_pcm24 seems to work!")
    plot(sig)

32-bit float files can be used since version 0.13 (see https://github.com/scipy/scipy/commit/ccbdff8).

In [ ]:
fs, sig = wavfile.read('data/test_wav_float32.wav')
plot(sig);

32-bit float files in WAVEX format, however, still don't work. They don't raise an error (maybe a warning) but yield wrong data (because int32 is used instead of float32).

In [ ]:
fs, sig = wavfile.read('data/test_wavex_float32.wav')
plot(sig);

Wow! That's not supposed to look like that!

## Writing

That's easy, you just need a NumPy array with an appropriate data type where each column represents a channel of audio data:

In [ ]:
data = array([[1.0,  -1.0],
              [0.75, -0.75],
              [0.5,  -0.5],
              [0.25, -0.25]], dtype='float32')

wavfile.write('scipy_float32.wav', 44100, data)

If you want to save 16-bit PCM files, you have to provide the NumPy array with data type `'int16'`. You can use my function `float2pcm()` from [`utility.py`](https://github.com/mgeier/python-audio/blob/master/audio-files/utility.py) to do the conversion:

In [ ]:
from utility import float2pcm

wavfile.write('scipy_pcm16.wav', 44100, float2pcm(data, 'int16'))

That's it! If you want a 32-bit PCM file, you can use `'int32'` instead of `'int16'`.

## Epilogue

Finally, let's see what versions we were using:

In [ ]:
import scipy
import IPython
print("Versions: SciPy = {}; NumPy = {}; IPython = {}".format(scipy.__version__, numpy.__version__, IPython.__version__))

import sys
print("Python interpreter:")
print(sys.version)